In [76]:
import pandas as pd
from imdb import IMDBDataset
import seaborn as sns
import numpy as np

# Points of interest

- energy / synapse (fJ) (energy / param)
- energy / input vector (μJ)
- Power (W)
- Throughput (vectors / s)
- Power / synapse (mW) (power / param)
- Synapses (params)
- Dataset size
- Training time (s)
- Epochs

In [48]:
imdb_dataset = IMDBDataset()
imdb_size = len(imdb_dataset)
imdb_size

25000

In [88]:
mnist_size = 60000

In [77]:
def load_benchmark_df(filename):
    df = pd.read_csv(filename)
    df = df.sort_values(by=["run", "epoch", "elapsed_time"])
    df["iteration_time"] = (
        df.groupby(["run", "epoch"])["elapsed_time"].diff().fillna(0)
    )
    df["energy_J"] = df["power_consumption"] * df["iteration_time"]
    return df

def get_summary_df(df, dataset_size):
    results = []
    for (run,), run_df in df.groupby(["run"]):
        epoch_group = run_df.groupby(["epoch"])
        epoch_mean_power = epoch_group["power_consumption"].mean()
        mean_power = epoch_mean_power.mean()
        train_duration = epoch_group["elapsed_time"].max().sum()
        epochs = len(epoch_group)
        params = run_df.total_params.iloc[0]
        power_per_param = mean_power * 1000 / param_count
        run_energy = epoch_group["energy_J"].sum().sum()
        iterations = len(run_df)
        results.append({
            "run": run,
            "mean_power_W": mean_power,
            "train_duration_s": train_duration,
            "epochs": epochs,
            "params": params,
            "power_per_param_mW": power_per_param,
            "energy_J": run_energy,
            "iterations": iterations,
        })
        
    summary = pd.DataFrame(results)

    summary["energy_per_param_fJ"] = summary["energy_J"] * 1e15 / summary["iterations"]
    summary["dataset_size"] = dataset_size
    summary["throughput"] = dataset_size / summary["train_duration_s"]
    summary["energy_per_vector_mJ"] = summary["power_per_param_mW"] / summary["throughput"] * 1000

    return pd.DataFrame([summary.drop(columns=["run"]).mean()])

# Summaries

In [89]:
filename = "benchmarks/lstm_benchmark_cuda.csv"
df = load_benchmark_df(filename)
lstm_df = get_summary_df(df, imdb_size)
lstm_df

,mean_power_W,train_duration_s,epochs,params,power_per_param_mW,energy_J,iterations,energy_per_param_fJ,dataset_size,throughput,energy_per_vector_mJ
0,37.055828,107.596667,5.0,4828930.0,0.008882,3974.6645,1955.0,2.033076e+15,25000.0,232.354267,0.038212


In [90]:
filename = "benchmarks/transformer_benchmark_cuda.csv"
df = load_benchmark_df(filename)
transformer_df = get_summary_df(df, imdb_size)
transformer_df

,mean_power_W,train_duration_s,epochs,params,power_per_param_mW,energy_J,iterations,energy_per_param_fJ,dataset_size,throughput,energy_per_vector_mJ
0,42.057901,90.03,5.0,4172034.0,0.010081,3779.958967,1955.0,1.933483e+15,25000.0,277.691844,0.036303


In [91]:
filename = "benchmarks/resnet18_benchmark_cuda.csv"
df = load_benchmark_df(filename)
resnet18_df = get_summary_df(df, mnist_size)
resnet18_df

,mean_power_W,train_duration_s,epochs,params,power_per_param_mW,energy_J,iterations,energy_per_param_fJ,dataset_size,throughput,energy_per_vector_mJ
0,19.120147,642.43,5.0,11181642.0,0.004583,12284.4951,4690.0,2.619295e+15,60000.0,93.395474,0.049091


In [96]:
lstm_df.to_excel("lstm_4060.xlsx", index=False)
transformer_df.to_excel("transformer_4060.xlsx", index=False)
resnet18_df.to_excel("resnet18_4060.xlsx", index=False)

In [97]:
lstm_df["energy_J"] / lstm_df["params"]

0    0.000823
dtype: float64